In [1]:
import http.client, urllib.request, urllib.parse, urllib.error, base64
from utils import *

In [2]:
headers = {
    # Request headers
    'Ocp-Apim-Subscription-Key': '5625f73035eb4558b9e6bcddea45edda',
}

########### get reporting_economy #############


In [3]:
params = urllib.parse.urlencode({
    # Request parameters
    'ig': 'individual',
    'reg': 'all',
    'gp': 'all',
    'lang': '1',
})

try:
    conn = http.client.HTTPSConnection('api.wto.org')
    conn.request("GET", "/timeseries/v1/reporters?%s" % params, "{body}", headers)
    response = conn.getresponse()
    data = response.read()
    print(data[0:100])
    json_data=data.decode('utf8')
    data=json.loads(json_data)
    
    conn.close()
except Exception as e:
    print("[Errno {0}] {1}".format(e.errno, e.strerror))



b'[{"code":"004","iso3A":"AFG","name":"Afghanistan","displayOrder":10},{"code":"008","iso3A":"ALB","na'


In [9]:
type(data[0]['code'])

str

In [3]:
countries_code=getDF(data,dtype=str)
countries_code.to_csv('./data/countries_code.csv',index=False)

NameError: name 'data' is not defined

### 获取 双边贸易数据###

In [3]:
countries_code=pd.read_csv('./data/countries_code.csv',dtype=str)
countries_code.head()

,code,iso3A,name,displayOrder
0,004,AFG,Afghanistan,10
1,008,ALB,Albania,40
2,012,DZA,Algeria,50
3,016,ASM,American Samoa,70
4,020,AND,Andorra,90


In [4]:
reporting_economies=countries_code['code'].tolist()

reporting_economies_str=""
for i in reporting_economies:
    reporting_economies_str+=i+","
reporting_economies_str=reporting_economies_str[0:-1]


In [5]:
partner_economies=countries_code['code'].tolist()
partner_economies_str=""
for i in partner_economies:
    if i in ['532','666']:
        continue
    partner_economies_str+=i+","
partner_economies_str=partner_economies_str[0:-1]


In [10]:
meta_data=Timeseries_datapoints_API(headers,reporting_economies_str,partner_economies_str,meta='True',max=1,timePeriod=timePeriod)

In [11]:
meta_data

{'title': 'One or more validation errors occurred.',
 'status': 400,
 'errors': {'i': ['The Indicator code is required']}}

In [12]:
timePeriods=['2016','2017']
for timePeriod in timePeriods:
    print(f'getting {timePeriod} data')
    data=get_data(timePeriod,headers,reporting_economies_str,partner_economies_str)
    df=getDF(data,dtype=str)
    df.to_csv(f'./data/data_{timePeriod}.csv',index=False)

getting 2018 data


KeyError: 'Summary'

## 获取products/sectors info

In [7]:
params = urllib.parse.urlencode({
    # Request parameters
    'pc': 'MT2',
    'lang': '1',
})

try:
    conn = http.client.HTTPSConnection('api.wto.org')
    conn.request("GET", "/timeseries/v1/products?%s" % params, "{body}", headers)
    response = conn.getresponse()
    data = response.read()
    json_data=data.decode('utf8')
    data=json.loads(json_data)
    conn.close()
except Exception as e:
    print("[Errno {0}] {1}".format(e.errno, e.strerror))

df=getDF(data,dtype=str)
df.to_csv('./data/ProductOrSectorCode_detail.csv',index=False)

In [8]:
df.head()

,code,name,note,productClassification,codeUnique,displayOrder,hierarchy
0,01,Animal products,None,MT2,MT2 - 01 -,None,01
1,02,Dairy products,None,MT2,MT2 - 02 -,None,02
2,03,"Fruits, vegetables, plants",None,MT2,MT2 - 03 -,None,03
3,04,"Coffee, tea",None,MT2,MT2 - 04 -,None,04
4,05,Cereals and preparations,None,MT2,MT2 - 05 -,None,05
